# Data Search and Download

In [ ]:
import astropy.units as u

from sunpy.net import Fido, attrs as a


## Overview of the `Fido` Unified Downloader

* Fido is sunpy's interface for searching and downloading solar physics data.
* It offers a unified interface for searching and fetching data irrespective of the underlying client or webservice from where the data is obtained.
* Offers a way to search and accesses multiple instruments and all available data providers in a single query.
* It supplies a single, easy, consistent and *extendable* way to get most forms of solar physics data the community need 

Fido offers access to data available through:

 * **VSO**
 * **JSOC**
 * **Individual data providers** from web accessible sources (http, ftp, etc)
 * **HEK**
 * **HELIO**
 
As described here Fido provides access to many sources of data through different `clients`, these clients can be defined inside sunpy or in other packages.
Lets print the current list of available clients within sunpy.

In [ ]:
Fido

You can list all of the supported instruments as well

In [ ]:
a.Instrument

## Searching for Data

Sunpy uses specified *attributes* to search for data using Fido. The range of these attributes is located in the `attrs` submodule. These `attr` parameters can be combined together to construct data search queries, such as searching over a certain time period, for data from a certain instrument with a certain wavelength etc.

Different clients and provides will have client-specific attributes, but the core attributes are:

* `a.Time`
* `a.Instrument`
* `a.Wavelength`

Let's use these different attributes to construct a query for our CME observation.

In [ ]:
cme_start = "2022-03-28T11:00"
cme_end = "2022-03-28T14:00"

In [ ]:
cme_time = a.Time(cme_start, cme_end)

We can inspect the instrument attribute to see what instrument `attrs` are currently supported through sunpy. Here we can see the instrument name (i.e. the name to be passed to the `a.Instrument` attribute, the client from which the data is available to access, and the full name of the instrument.

In [ ]:
a.Instrument

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>Fido search attributes also support autcomplete. Try using the autcomplete feature in the notebook to look for your favorite instrument.</h3>
</div>

We can combine our time and instrument attributes to search for AIA data within our selected time range using `Fido.search`

In [ ]:
Fido.search(cme_time & a.Instrument.aia)

We can further filter our results using the `Wavelength` search attribute.

In [ ]:
Fido.search(cme_time & a.Instrument.aia & a.Wavelength(304*u.angstrom))

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>We want to query the AIA data at a 5 minute cadence rather than the full 12 second cadence. How would we modify our above query to accomplish this?</h3>
</div>

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>We've written a query for the AIA data above. How would we write a query for EUVI data from STEREO-A for the same time range, cadence, and wavelength?</h3>
</div>

### Combining Queries

In addition to making queries for individual instruments, we can also logically combine queries for multiple instruments at once. For example, if we wanted to search for data from both AIA and SECCHI for the same time range and passband,

In [ ]:
Fido.search(cme_time, a.Instrument.aia | a.Instrument.secchi, a.Wavelength(304*u.angstrom), a.Sample(5*u.minute))

What if we also wanted to look for the GOES XRS data during this same interval?
GOES/XRS data does not have a `Wavelength` or `Sample` associated with it, but we can still combine the queries for all three of these instruments.

In [ ]:
aia_or_secchi = (a.Instrument.aia | a.Instrument.secchi) & a.Wavelength(304*u.angstrom) & a.Sample(5*u.minute)

In [ ]:
goes_query = a.Instrument.xrs & a.goes.SatelliteNumber(17)

In [ ]:
combined_query = Fido.search(cme_time, aia_or_secchi | goes_query)

In [ ]:
combined_query

### Using External `Fido` Clients and Post-search filtering

Within `sunpy` core, we support a number of clients to common data providers.
However, the `Fido` search interface is extensible such that external packages can write that their own clients that extend `Fido` in order to additional data sources.
One such example is the `sunpy_soar` package which adds a client for the Solar Orbter Archive (SOAR) which is located here at ESAC!

In [ ]:
import sunpy_soar

Note that after importing `sunpy_soar`, the SOAR is now listed as a client that `Fido` will search.

In [ ]:
Fido

We can now search for level 2 data from the 304 angstrom channel of the EUI instrument on SolO.
Let's attach one last query to our combined query from above.

In [ ]:
eui_query = a.Level(2) & a.soar.Product('EUI-FSI304-IMAGE')

In [ ]:
combined_query = Fido.search(cme_time, aia_or_secchi | goes_query | eui_query)

Now, we have a total of 4 results from our 4 different instruments for the same time interval.

In [ ]:
combined_query

## Downloading Data

We can easily make a single download request from all of our different clients by passing in our combined query for AIA, EUVI, EUI and XRS.

In [ ]:
files = Fido.fetch(combined_query, path='data/{instrument}')

In [ ]:
files

In [ ]:
Fido.fetch(files)